# Trabalho Prático 3 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## *Esquemas pós-quânticos de assinaturas digitais* - qTESLA

Nota: Baseado no documento qTESLA (https://qtesla.org/wp-content/uploads/2020/04/qTESLA_round2_14.04.2020.pdf)

### Descrição base da implementação

Dos três esquemas pós-quânticos de assinaturas digitais propostos no enunciado do trabalho (*qTesla*, *Dilithium* e *Rainbow*), um dos que escolhemos fazer foi o **qTESLA**. O **qTESLA** é um esquema *high algebra* de assinaturas digitais da classe **LWE**(*Learning with Errors*). Recorde-se que no **RLWE** a informação privada é um par *(s,e)*, ambos pequenos polinómios, com *s* gerado aleatoriamente e *e* gerado numa distribuição gaussiana discreta *X*. A chave pública é o par *(a,b)* que verifica  *b == a * s + e mod f mod q*. De acordo com o documento mencionado acima, aquele que tentamos implementar foi o qTESLA-I, que se baseia numa geração de parâmetros heurística.

De seguida, tentaremos explicar unicamente as três principais funções que devem ser chamadas nesta classe: **geraChaves()**, **assina()** e **verifica()**.

**Geração da chave pública *(t,a)* e da chave privada (s, e, a) *(função geraChaves())***

  - O **a** é gerado a através do anél **Rq** de forma aleatória;
  - Depois, escolhe-se um **s** que pertença a **R** (usando uma distribuição gaussiana), processo que se repete enquanto a soma das *h* maiores entradas for superior ao parâmetro *LS*;
  - A seguir, escolhe-se também um **e** que pertença a **R**, processo que se repete enquanto a soma das *h* maiores entradas for superior a *LE*;
  - Depois, calcula-se **t = a * s + e** pertencente a **R**;;
  - Por fim, são retornadas a chaves privada **sk=(s, e, a)** e a chave pública **pk=(t, a)**.
  
**Assinatura *(função assina())***

  - Recebe como argumentos a mensagem a assinar e a chave privada;
  - Primeiro, **y** é gerado aleatóriamente seguindo uma distribuição uniforme com coeficientes curtos (*B-short*) em **Rq**;
  - De seguida, **v** é calculado através da componente **a** da chave privada *sk* multiplicando com o **y** gerado anteriormente, sendo que como **v** têm de estar no intervalo [-q/2,q/2-1], este resultado é ainda sujeito ao respetivo arredondamento;
  - Depois, calcula-se um **c'**(ou *c1*) que é o resultado do hash de **v** com o hash da mensagem;
  - A seguir, procede-se ao encoding de **c'** para obtermos um **c**.
  - E por fim, calcula-se **z** através da expressão: *z = y + s*c*, sendo que o resultado da assinatura é o par *(z,c')*.
  

**Verificação *(função verifica())***

  - Recebe como argumentos a mensagem original, a assinatura digital a verificar e a chave pública;
  - Inicialmente, começa por calcular o encoding **c** através da componente **c'** da assinatura digital;
  - Depois, calcula-se **w** através da expressão: *w = a*z - t*c* pertencente a **Rq**, sendo que este resultado também deve ser arredondado para ficar entre [-q/2,q/2-1];
  - A seguir, verifica-se se **c'** é diferente do hash de **v** com o hash da mensagem, pelo que se for retorna -1 e a verificação falha, caso contrário retorna 0 e a assinatura é válida.
  

In [3]:
import numpy as np
import hashlib

# Baseado no esquema da página 10-14 do documento https://qtesla.org/wp-content/uploads/2020/04/qTESLA_round2_14.04.2020.pdf

class qTESLA(object):
    
    def __init__(self, K, k, h, Le, Ls, B, d, n, q, timeout=None):
        
        # Parametros iniciais
        self.K = K
        self.k = k
        self.h = h
        self.Le = Le
        self.Ls = Ls
        self.B = B
        self.d = d
        self.n = n
        self.q = q
        
        # Definicao dos Aneis necessarios
        Zx.<x>  = ZZ[]
        self.Zx = Zx
        Gq.<z>  = GF(q)[]
        
        # R is the quotient Ring Z[x]/<x^n + 1> (referido no docuemnto, mas nao necessario)
        R.<x> = Zx.quotient(x^n+1)  # R
        self.R = R
        # Rq is the quotient Ring Zq[x]/<x^n + 1>
        Rq.<z> = Gq.quotient(z^n+1) # R/q
        self.Rq = Rq
    
    
    # Faz o Hash de uma determinada mensahem em SHA3_256
    def Hash(self, s):
        
        h = hashlib.sha3_256()
        h.update(s)
        return h.digest() 
    
    
    # Gera o parametro a aleatoriamente de Rq
    def GeraA(self):
        
        return self.Rq.random_element()
    
    
    # Produz o polinomio f modulo q. Mas, em vez de ser entre 0 e q-1, fica entre -q/2 e q/2-1
    def arredonda_mod(self, f):
        
        g = list(((elem + self.q//2) % self.q) - self.q//2 for elem in f.list())
        return self.Zx(g)
    
    
    # checkS: simplifica a redução de segurança, garantindo que ||sc|| <= S
    def checkS(self, s):
        
        total = 0
        ls = list(s)
        ls.sort(reverse=True)
        for i in range(0, self.h):
            total += ls[i]
        if total > self.Ls:
            return 1
        return 0
    
    
    # checkE: garante a correção do esquema, verificando se ||ec|| <= S
    def checkE(self, e):
        
        total = 0
        ls = e.list()
        ls.sort(reverse=True)
        for i in range(0, self.h):
            total += int(ls[i])
        if total > self.Le:
            return 1
        return 0
    
    
    # Hash-based function H
    # Recebe o polinomio v € Rq e o hash da mensagem G(m)
    # Retorna c' € {0,1}^K
    def H(self, v, h_message):
        
        w = [0] * self.n
        for i in range(self.n):
            # val <- v_i,j mod 2^d
            val = v[i] % 2^self.d
            if val > 2^(self.d-1):
                # val <- val - 2^d
                val = val - 2^self.d
            # w(i-1)-n+j <- (v_i,j - val)/2^d
            w[i] =  (v[i] - val)/2^self.d
        return self.Hash(str(w).encode() + h_message)
    
    
    # Encoding function, Enc
    # Recebe c' € {0,1}^K
    # Retorna os arrays pos_list € {0, ..., n-1}^h e o sign_list € {-1,1}^h contendo as posições
    # e sinais, respectivamente, dos elementos diferentes de zero de c € H_n,h
    def Enc(self, c1):
        
        # D <- 0, cnt <- 0
        D = 0
        cnt = 0
        # rateXOF is the SHAKE128 rate constant 168
        rateXOF = 168
        # r <- cSHAKE128(c', rateXOF, D)
        r = hashlib.shake_128(c1 + bytes(D)).digest(int(rateXOF))
        i = 0
        pos_list = [None] * self.h
        sign_list = [None] * self.h
        # Set all coefficients of c to 0
        c = [0] * self.n
        while i < self.h:
            # if cnt > (rateXOF - 3)
            if cnt > rateXOF-3:
                # D <- D + 1; cnt <- 0
                D += 1
                cnt = 0
                # r <- cSHAKE128(c', rateXOF, D)
                r = hashlib.shake_128(c1 + bytes(D)).digest(int(rateXOF))
            # pos <- (r_cnt . 2^8 + r_cnt+1) mod n
            pos = (r[cnt] * 2^8 + r[cnt+1]) % self.n
            if c[pos] == 0:
                # if r_cnt+2 mod 2 == 1
                if r[cnt+2] % 2 == 1:
                    # c_pos <- -1
                    c[pos] =  -1
                else:
                    # c_pos <- 1
                    c[pos] = 1
                pos_list[i] = pos
                sign_list[i] = c[pos]
                i += 1
            cnt += 3
        return (pos_list, sign_list)
    
    
    # Gera um polinomio no anel Rq dada a lista de posicoes e a lista de sinais(positivo ou negativo)
    def geraPol(self, pos_list, sign_list):
        
        r = [0]*512
        j = 0
        for i in pos_list:
            r[i] = sign_list[j]
            j+=1
        return self.Rq(r)
    
    
    # Geração de chaves publica e privada
    # secret key sk = (s, e, a), 
    # and public key pk = (t, a)
    def geraChaves(self):
        
        # a1, ..., ak <- GenA()
        a = self.GeraA()
        s = None
        while (True):
            # s <- R (GaussSampler(seed_s, counter))
            s = self.Rq.random_element(distribution="gaussian")
            # checkS(s) != 0
            if self.checkS(s) == 0:
                break
        
        e = None
        while (True):
            # e <- R (GaussSampler(seed_e, counter))
            e = self.Rq([abs(ZZ.random_element(x=self.Le // 60, distribution="gaussian")) for _ in range(512)])
            # checkE(e) != 0
            if self.checkE(e) == 0:
                break
        
        # t <- a*s + e mod q
        t = a*s + e
        sk = (s, e, a)
        pk = (t, a)
        return (sk, pk)
    
    
    # Geração da assinatura digital
    # retorna a assinatura (z, c1)
    def assina(self, message, sk):
        
        # Sampling of y <- Generate values uniformly in range [-B,B] 
        y = self.Rq.random_element(x=-self.B, y=self.B+1, distribution="uniform")
        # for i = 1,..,k do vi = ai*y mod+-q (ou seja, necessita de arredondamento)
        v = self.arredonda_mod(sk[2]*y)
        # c' <- H(v1, ...,vk, G(m))
        c1 = self.H(v, self.Hash(str(message).encode()))
        # c = {pos_list, sign_list} <- Enc(c')
        c = self.Enc(c1)
        # z <- y + sc
        z = y + sk[0] * self.geraPol(c[0],c[1])
        # return (z, c')
        return (z, c1)
    
    
    # Verificação da assinatura digital
    # {0,-1} -> accept, reject signature
    def verifica(self, message, signature, pk):
        
        # c = {pos_list, sign_list} <- Enc(c')
        c = self.Enc(signature[1])
        # w <- a*z - t*c mod+-q (ou seja, necessita de arredondamento)
        w = self.arredonda_mod(pk[1]*signature[0] - pk[0]*self.geraPol(c[0],c[1]))
        # c' != H(w1, ...,wk, G(m))
        if signature[1] != self.H(w, self.Hash(str(message).encode())):
            return -1
        return 0
        

#### Testagem da classe definida acima:

In [4]:
import random, string

# Parametros iniciais (adaptamos a seguranca qTESLA-p-I para que fosse mais rapido)
K = 256
# number of public polynomials a1, ..., ak (só vamos usar um neste caso)
k = 1
# number of nonzero entries of output elements of Enc 
h = 30
# bound in checkE
Le = 1586
# bound in checkS
Ls = 1586
# determines interval the randomness is chosen from during signing
B = 2^19-1
# number of rounded bits
d = 21
# dimension
n = 512
# modulus q == 1 mod 2n, q > 2^d+1 + 1 (como 2^d -1 == 4194305, então q pode ser o seguinte)
q = 4205569

# Teste
print("[Testagem da classe acima]:\n")

qtesla = qTESLA(K, k, h, Le, Ls, B, d, n, q)
# Inserir uam mensagem a assinar
message = input("Insira uma mensagem a assinar: ")

# Gerar as Chaves
(sk, pk) = qtesla.geraChaves()
# Assinar a mensagem com a chave privada
s = qtesla.assina(message, sk)
#print(s)
# Verificar a assinatura com a chave publica
res = qtesla.verifica(message, s, pk)

if res == 0:
    print("A assinatura digital é válida!!")
else:
    print("A assinatura digital não é válida!!")

[Testagem da classe acima]:

Insira uma mensagem a assinar: Viva a Estruturas Criptográficas!!!
A assinatura digital é válida!!
